In [3]:
import pandas as pd
import numpy as np

In [4]:
def get_z_score(series):
    # 12개월 누적 로그 변화율 계산
    log_return = np.log(series.astype(float)).diff(12)
    # Z-score 산출
    return (log_return - log_return.mean()) / log_return.std()

In [6]:
# 소매판매액지수 처리
try:
    df_retail_raw = pd.read_csv('data/소매판매액지수raw.csv', encoding='cp949')
except:
    df_retail_raw = pd.read_csv('data/소매판매액지수raw.csv', encoding='utf-8-sig')

In [7]:
df_retail = pd.DataFrame({
    'retail': df_retail_raw.iloc[1, 1:].values
}, index=df_retail_raw.columns[1:])
df_retail.index = pd.to_datetime(df_retail.index.str.replace(r'[^\d.]', '', regex=True), format='%Y.%m')
df_retail['retail_z'] = get_z_score(df_retail['retail'])

In [10]:
# 신용카드사용액 처리
try:
    df_card_raw = pd.read_csv('data/신용카드사용액raw.csv', encoding='cp949')
except:
    df_card_raw = pd.read_csv('data/신용카드사용액raw.csv', encoding='utf-8-sig')
df_card = pd.DataFrame({
    'card': df_card_raw.iloc[0, 2:].values
}, index=df_card_raw.columns[2:])
df_card.index = pd.to_datetime(df_card.index, format='%Y/%m')
df_card['card_z'] = get_z_score(df_card['card'])

In [11]:
# 전산업생산지수 처리
try:
    df_prod_raw = pd.read_csv('data/전산업생산지수raw.csv', encoding='utf-8')
except:
    df_prod_raw = pd.read_csv('data/전산업생산지수raw.csv', encoding='cp949')

df_prod = pd.DataFrame({
    'production': df_prod_raw.iloc[0, 1:].values
}, index=df_prod_raw.columns[1:])
df_prod.index = pd.to_datetime(df_prod.index.str.replace(r'[^\d.]', '', regex=True), format='%Y.%m')
df_prod['prod_z'] = get_z_score(df_prod['production'])

In [12]:
# 데이터 통합 및 종합 지수(평균) 계산
combined = df_retail[['retail_z']].join(df_card[['card_z']], how='inner')
combined = combined.join(df_prod[['prod_z']], how='inner')

In [13]:
# 세 지표의 평균을 종합 실물 지수로 정의
combined['real_index'] = combined.mean(axis=1)

In [14]:
# 컬럼명 정리 및 저장
final_df = combined.reset_index().rename(columns={
    'index': 'date',
    'retail_z': 'retail',
    'card_z': 'card',
    'prod_z': 'production'
})

In [15]:

final_df.to_csv('data/(창민)실물지표계산.csv', index=False, encoding='utf-8-sig')
print("종합 실물지표 파일이 'data/(창민)실물지표계산.csv'로 저장되었습니다.")

종합 실물지표 파일이 'data/(창민)실물지표계산.csv'로 저장되었습니다.
